In [55]:
using CSV, DataFrames, Statistics, Gurobi, JuMP, Dates, CategoricalArrays

In [56]:
data = CSV.read("KaggleV2-May-2016.csv", DataFrame)

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
,Float64,Int64,String1,String31,String31,Int64,String31,Int64,Int64,Int64,Int64,Int64,Int64,String3
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,No
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,Yes
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,Yes
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,No


In [57]:
# 1. Parse the datetime strings (they have a trailing "Z", so strip it)
data.ScheduledDT   = DateTime.(replace.(data.ScheduledDay, "Z" => ""), dateformat"yyyy-mm-ddTHH:MM:SS")
data.AppointmentDT = DateTime.(replace.(data.AppointmentDay, "Z" => ""), dateformat"yyyy-mm-ddTHH:MM:SS")

# 2. Keep only the date part (drop time-of-day)
data.ScheduledDate   = Date.(data.ScheduledDT)
data.AppointmentDate = Date.(data.AppointmentDT)

# 3. Compute waiting time in days (scheduled - appointment)
data.lead_time = Dates.value.(data.AppointmentDate .- data.ScheduledDate)
data

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time
,Float64,Int64,String1,String31,String31,Int64,String31,Int64,Int64,Int64,Int64,Int64,Int64,String3,DateTime,DateTime,Date,Date,Int64
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,No,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,Yes,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,Yes,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,No,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0


In [58]:
# Only make truly categorical variables categorical
data.Neighbourhood = categorical(data.Neighbourhood)
data.Gender = categorical(data.Gender)
data.Handcap = categorical(data.Handcap) 

110527-element CategoricalArray{Int64,1,UInt32}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [59]:
#turn No-show into binary
data."No-show" = ifelse.(data."No-show" .== "No", 0, 1)

110527-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 1
 0
 ⋮
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [60]:
#filter ages between 0 and 100
data = filter(row -> (row.Age >= 0) && (row.Age <= 100), data)

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time
,Float64,Int64,Cat…,String31,String31,Int64,Cat…,Int64,Int64,Int64,Int64,Cat…,Int64,Int64,DateTime,DateTime,Date,Date,Int64
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0


In [61]:
# Filter out negative lead times (data quality issue)
data = filter(row -> row.lead_time >= 0, data)
println("After filtering lead_time >= 0: ", nrow(data), " rows")

After filtering lead_time >= 0: 110514 rows


In [62]:
neigh_sizes = combine(groupby(data, :Neighbourhood), nrow => :count)
neighbourhoods_to_keep = filter(row -> row.count >= 55, neigh_sizes).Neighbourhood
data = filter(row -> row.Neighbourhood in neighbourhoods_to_keep, data)


Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time
,Float64,Int64,Cat…,String31,String31,Int64,Cat…,Int64,Int64,Int64,Int64,Cat…,Int64,Int64,DateTime,DateTime,Date,Date,Int64
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0


In [63]:
#drop duplicate rows
data = unique(data)

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time
,Float64,Int64,Cat…,String31,String31,Int64,Cat…,Int64,Int64,Int64,Int64,Cat…,Int64,Int64,DateTime,DateTime,Date,Date,Int64
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0


In [64]:
#create day of week and appointment month features
data.day_of_week = dayname.(data.AppointmentDate)
data.appointment_month = monthname.(data.AppointmentDate)

#turn day of week and appointment month into categorical variables
data.day_of_week = categorical(data.day_of_week, ordered=true, levels=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
data.appointment_month = categorical(data.appointment_month, ordered=true, levels=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"])
data

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time,day_of_week,appointment_month
,Float64,Int64,Cat…,String31,String31,Int64,Cat…,Int64,Int64,Int64,Int64,Cat…,Int64,Int64,DateTime,DateTime,Date,Date,Int64,Cat…,Cat…
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2,Friday,April
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2,Friday,April
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2,Friday,April
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April


In [65]:
data.Gender_binary = ifelse.(data.Gender .== "F", 1, 0)

110458-element Vector{Int64}:
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 1
 ⋮
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [66]:
using Random
Random.seed!(42)  # For reproducibility

# Create random indices
n = nrow(data)
train_indices = shuffle(1:n)[1:Int(floor(0.8 * n))]
test_indices = setdiff(1:n, train_indices)

train_data = data[train_indices, :]
test_data = data[test_indices, :]

using Statistics

# Standardize only Age and lead_time
train_age_mean = mean(train_data.Age)
train_age_std = std(train_data.Age)
train_lead_mean = mean(train_data.lead_time)
train_lead_std = std(train_data.lead_time)

train_data.Age_std = (train_data.Age .- train_age_mean) ./ train_age_std
train_data.lead_time_std = (train_data.lead_time .- train_lead_mean) ./ train_lead_std

test_data.Age_std = (test_data.Age .- train_age_mean) ./ train_age_std
test_data.lead_time_std = (test_data.lead_time .- train_lead_mean) ./ train_lead_std


22092-element Vector{Float64}:
 -0.667496337024902
 -0.5370556344053339
 -0.667496337024902
 -0.5370556344053339
 -0.6022759857151179
 -0.5370556344053339
 -0.667496337024902
 -0.667496337024902
 -0.667496337024902
 -0.667496337024902
 -0.08051317523684559
 -0.6022759857151179
 -0.4066149317857658
  ⋮
  1.8108770127468916
 -0.5370556344053339
 -0.667496337024902
 -0.5370556344053339
 -0.667496337024902
 -0.667496337024902
 -0.667496337024902
 -0.667496337024902
 -0.667496337024902
 -0.667496337024902
 -0.667496337024902
  2.0065380666762436

In [67]:
# Export training and test sets
CSV.write("MLOpt_train_data.csv", train_data)
CSV.write("MLOpt_test_data.csv", test_data)

"MLOpt_test_data.csv"